# 프로젝트 파일 구조 및 기능 설명
---

## 폴더 구조
```
medicine/
├── configs/            # 경로 설정
├── preprocessing/      # 데이터 전처리
├── dataset/            # 데이터셋 클래스
├── model/              # 모델 정의, 학습, 추론
├── runner/             # 통합 실행 스크립트
├── data/raw/           # 원본 데이터
├── outputs/            # 학습 결과 저장
└── submit/             # 제출 CSV 저장
```

## 실행 순서

### YOLO 파이프라인
```
coco_data_integration → coco_validation → yolo_converter → yolov8s_model → yolo_inference
```

### Faster R-CNN 파이프라인
```
coco_data_integration → rcnn_model → rcnn_train → rcnn_inference
```

### runner로 한번에 실행
```bash
python runner/main_yolo.py   # YOLO 변환 + 학습 + 추론
python runner/main_rcnn.py   # RCNN 학습 + 추론
```

---
## 공통 파일

### `imports.py`
- 프로젝트 전체에서 사용하는 라이브러리 모음 (torch, pandas, numpy, YOLO, tqdm 등)
- 각 .py에서 `from imports import *`로 불러옴
- **직접 수정할 일 없음**

### `configs/paths.yml`
- 모든 경로를 한 곳에서 관리하는 설정 파일
- data, dataset, model, outputs, submit 경로 정의
- **경로 변경 시 여기만 수정**

### `configs/load_paths.py`
- paths.yml을 읽어서 절대경로로 변환
- `DATA_RAW`, `DATASET_YOLO`, `PROJECT_ROOT` 등 상수 제공
- **직접 수정할 일 없음**

---
## 전처리 (preprocessing/)

### `coco_data_integration.py`
- 분산된 JSON 정답지를 이미지별로 통합
- `get_integrated_coco_data()` - YOLO, RCNN 모두 사용
- 한 번 실행 후 캐시 저장 → 다음부터 빠르게 로드
- **직접 수정할 일 없음**

### `coco_validation.py`
- 데이터 품질 검증 (박스 오류, 누락 검사)
- 선택 사항. 데이터 확인용
- **직접 수정할 일 없음**

### `data_inventory_check.py`
- 데이터 현황 분석 (클래스 분포, 이미지 개수)
- 선택 사항. 데이터 확인용
- **직접 수정할 일 없음**

### `yolo_converter.py`
- COCO 포맷 → YOLO 포맷 변환 + train/val 8:2 계층화 분할
- `dataset/yolo_dataset/`에 이미지, 라벨, data.yaml 생성
- **처음 1회만 실행** (이미 있으면 재사용 선택)
- 수정할 부분: `train_ratio=0.8` (분할 비율 바꾸고 싶을 때만)

---
## YOLO 파일

### `model/yolov8s/yolov8s_model.py` - 학습
- `train_model(model_obj)` - YOLO 모델 객체를 받아서 학습
- 하이퍼파라미터는 함수 내부에 하드코딩 (epochs, imgsz, batch, augmentation 등)
- 학습 결과 → `outputs/yolov8_learn_results/yolov8_pill_{timestamp}/`에 저장
- `log_save(model_obj, save_dir, metrics)` - 학습 조건 + 결과를 txt로 저장
- 로그 저장 경로: `outputs/logs/`
- **개별 실행 시** 54~69줄 train 파라미터 직접 수정

### `model/yolov8s/yolo_inference.py` - 추론 + CSV
- `find_best_model()` - `outputs/yolo/` 에서 **가장 최근 학습된** best.pt 자동 탐색
- ⚠️ 현재 학습 저장 경로(`yolov8_learn_results`)와 불일치 → 추후 수정 필요
- CSV 이름 자동 생성: `yolo1` → `submit/yolo1.csv`

### `runner/main_yolo.py` - 통합 실행
- ⚠️ 현재 `train_model` 시그니처 변경으로 호출 깨짐 → 추후 수정 필요

### YOLO 출력 경로
```
outputs/yolov8_learn_results/yolov8_pill_{timestamp}/
├── weights/best.pt      ← 최고 성능 모델
├── weights/last.pt      ← 마지막 에포크 모델
├── results.png          ← 학습 곡선 그래프
└── results.csv          ← 에포크별 수치

outputs/logs/
└── {timestamp}.txt      ← 학습 조건 + mAP 결과 로그

submit/yolo1.csv         ← 제출 CSV
```

---
## Faster R-CNN 파일

### `dataset/faster_rcnn/rcnn_dataset.py` - 데이터셋
- `PillDataset(split="train"/"val"/"test")` - 내부에서 8:2 분할
- YOLO와 달리 별도 변환 과정 없이 바로 사용
- **직접 수정할 일 없음**

### `model/faster_rcnn/rcnn_model.py` - 모델 정의
- `create_model(num_classes)` - ResNet50+FPN 백본
- **직접 수정할 일 없음**

### `model/faster_rcnn/rcnn_train.py` - 학습
- `run_training(batch_size, num_epochs, lr, name)` 함수
- 매 에포크마다 mAP 측정, best 모델 자동 저장
- **개별 실행 시** 286줄에서 파라미터 수정

### `model/faster_rcnn/rcnn_inference.py` - 추론 + CSV
- **가장 최근 학습된** `.pth` 자동 탐색 (`_last` 제외)
- CSV 이름 자동 생성: `rcnn1` → `submit/rcnn1.csv`
- **직접 수정할 일 없음**

### `runner/main_rcnn.py` - 통합 실행
- 학습 → 추론 순차 실행
- **팀원이 수정할 부분 (상단 17~29줄):**
```python
MODEL_NAME = "rcnn1"      # 실험 이름 (rcnn1, rcnn2, ...)
BATCH_SIZE = 4             # 배치 사이즈
NUM_EPOCHS = 2             # 에포크 수
LR = 0.01                  # 학습률
CONF_THRESHOLD = 0.25      # 추론 confidence

DO_TRAIN = True            # 학습
DO_INFERENCE = False       # 추론 + CSV
```

### R-CNN 출력 경로
```
outputs/rcnn/rcnn1.pth       ← 최고 성능 모델 (best)
outputs/rcnn/rcnn1_last.pth  ← 마지막 에포크 모델
submit/rcnn1.csv             ← 제출 CSV
```

---
## 실행 모드 (DO_XXX)

main 파일 상단의 `DO_XXX = True/False`로 원하는 단계만 선택:

| 예시 | DO_CONVERT | DO_TRAIN | DO_INFERENCE |
|------|:---:|:---:|:---:|
| 처음 학습 | True | True | False |
| 재학습 (변환 완료) | False | True | False |
| 추론만 | False | False | True |
| 전부 실행 | True | True | True |

---
## 개별 파일 실행

각 .py 파일은 독립적으로도 실행 가능:

```bash
# 전처리
python preprocessing/coco_data_integration.py
python preprocessing/coco_validation.py
python preprocessing/yolo_converter.py

# YOLO
python model/yolov8s/yolov8s_model.py    # 파라미터: 39줄에서 수정
python model/yolov8s/yolo_inference.py

# Faster R-CNN
python model/faster_rcnn/rcnn_train.py   # 파라미터: 286줄에서 수정
python model/faster_rcnn/rcnn_inference.py
```

---
## 추론 시 모델 자동 선택
- YOLO: `outputs/yolo/` 에서 **가장 최근 학습된** `best.pt` 자동 탐색
- RCNN: `outputs/rcnn/` 에서 **가장 최근 학습된** `.pth` 자동 탐색 (`_last` 제외)
- mAP 기준이 아닌 **수정 시간** 기준으로 최신 모델 선택